<a href="https://colab.research.google.com/github/genesio44/sharpe_ratio/blob/main/sharpe_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A EFICIÊNCIA DO ÍNDICE DE SHARPE COMO MEIO DE TOMADA DE DECISÃO NA ALOCAÇÃO DE PORTIFÓLIO
<p>O desejo, nem sempre alcançável, de todo investidor é obter o máximo de retorno com o mínimo risco. No intuito de satisfazer esse desejo, são criados os modelos de otimização de portifólio. O Índice de Sharpe, por exemplo,  é um indicador que avalia a rentabilidade de um investimento em relação ao risco de sua aplicação. </p>
<p>Neste projeto, atribuiremos alocações aleatórias para um dado grupo de ações pra encontrar a alocação que maximizou o Índice de Sharpe em determinado ano.
Em seguida, verificaremos se essa alocação consegue figurar entre as 5% melhores alocações para o ano seguinte, também de acordo com o Índice de Sharpe.</p>

#PACOTES NECESSÁRIOS

In [1]:
# Caso ainda não o possua instalado, instalar o yfinance

In [ ]:
pip install yfinance --upgrade --no-cache-dir

In [3]:
import yfinance as yf #Acessa as informações das ações no Yahoo Finance
from datetime import datetime #Realiza manipulações de data
from datetime import timedelta #Realiza cálculos de variação de tempo
from dateutil.relativedelta import relativedelta #Realiza o cálculo para variação mensal
import numpy as np #Realiza o processamento de diversas funções matemáticas
import pandas as pd #Realiza as alterações necessárias nos dataframes
from pandas_datareader import data
import matplotlib.pyplot as plt #Plota os gráficos, caso seja necessário
%matplotlib inline

#ENCONTRANDO O PORTIFÓLIO ÓTIMO

<p>Criamos uma lista com os códigos das ações que serão utilizadas no experimento (nesse caso, as ações que compõem o meu portifólio em 2021).</p>
<p>Vale lembrar que colocaremos o .SA ao final dos tickers, para adequar-se à nomeclatura utilizada pelo Yahoo Finance para as ações da bolsa brasileira.</p>

In [4]:
stocks = ['WEGE3.SA',
'BRML3.SA',
'LOGG3.SA',
'EQTL3.SA',
'HYPE3.SA',
'LREN3.SA',
'RAPT4.SA',
'BBAS3.SA',
'ITSA4.SA',
'QUAL3.SA',
'VULC3.SA',
'VVAR3.SA',
'MGLU3.SA',
'AZUL4.SA',
'LEVE3.SA',
'MDIA3.SA',
'EGIE3.SA'
]

<p>Definimos então uma fórmula que consiga nos fornecer uma grande quantidade de alocações possíveis, além dos respectivos retornos e volatilidades.</p>
<p>A ideia aqui é montar a fórmula padrão que nos retornará as informações para qualquer ano escolhido.</p>

In [13]:
def ports(ano):
  end = str(ano)+'-12-31' #Define a data final como dia 31 de dezembro do ano utilizado
  start = str(ano)+'-01-01' #Define a data inicial com dia 01 de janeiro do ano utilizado
  pf_data = pd.DataFrame() #Cria um dataframe
  for s in stocks:
    pf_data[s] = yf.Ticker(s).history(start=start,end=end)['Close'] #Cria um loop para obter as cotações de fechamento da ações da nossa lista durante o período
  cov_matrix = pf_data.pct_change().apply(lambda x: np.log(1+x)).cov() #Calcula a matriz de covariância entre essas ações
  ind_er = np.log(pf_data/pf_data.shift(1)).mean()*250 #O logaritmo das variações de preços nos dá o retorno esperado diário
  #multiplicando esse valor pela quantidade de dias úteis (250), obtem o retorno esperado anual
  p_ret = [] # Cria um intervalo vazio para o retorno esperado de cada portifólio
  p_vol = [] # Cria um intervalo vazio para a volatilidade ou risco de cada portifólio
  p_weights = [] # Cria um intervalo vazio para os pesos de cada ação no portifólio
  num_portfolios = 100000 #Define a quantidade de portifólios que desejamos testar
  for portfolio in range(num_portfolios): #Cria um loop para a obtenção de  portifólios
    weights = np.random.random(len(stocks)) #Define números aleatórios para os pesos de cada ação
    weights = weights/np.sum(weights) #Divide estes números por sua soma, para obter o valor aleatório em percentual
    p_weights.append(weights) #Adiciona ao respectivo intervalo
    returns = np.dot(weights, ind_er) # Obtém o retorno dos portifólios multiplicando o retorno esperado anual pelo peso de cada ação
    p_ret.append(returns) #Adiciona ao respectivo intervalo
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()#Calcula a variância dos portifólios
    sd = np.sqrt(var) # Obtem o desvio padrão através da raiz quadrada da variância
    ann_sd = sd*np.sqrt(250) #Multiplica o desvio padrão pela quantidade de dias úteis
    p_vol.append(ann_sd) #Adiciona ao respectivo intervalo
    data = {'Returns':p_ret, 'Volatility':p_vol}
  for counter, symbol in enumerate(pf_data.columns.tolist()):
      #print(counter, symbol)
      data[symbol] = [w[counter] for w in p_weights]
  portfolios  = pd.DataFrame(data)
  return portfolios

Utilizamos a fórmula que acabamos de definir para encontrar a relação de possíveis alocações para 2018:

In [14]:
base2018 = ports(2018)

Agora basta encontrar o portifólio que maximiza o Índice de Sharpe, o que podemos fazer da seguinte forma:

In [15]:

rf=0.01 #Define um valor de remuneração para o ativo livre de risco
opt2018 = base2018.iloc[((base2018['Returns']-rf)/base2018['Volatility']).idxmax()] #Encontra a alocação que maximiza a relação retorno risco
opt2018

Returns       0.488989
Volatility    0.103861
WEGE3.SA      0.071777
BRML3.SA      0.037744
LOGG3.SA      0.180270
EQTL3.SA      0.089863
HYPE3.SA      0.037589
LREN3.SA      0.083765
RAPT4.SA      0.010120
BBAS3.SA      0.003602
ITSA4.SA      0.038129
QUAL3.SA      0.019611
VULC3.SA      0.158847
VVAR3.SA      0.025941
MGLU3.SA      0.076976
AZUL4.SA      0.022535
LEVE3.SA      0.038274
MDIA3.SA      0.035370
EGIE3.SA      0.069587
Name: 74206, dtype: float64

#VERIFICANDO O DESEMPENHO PARA OS ANOS POSTERIORES

Criamos agora uma lista contendo apenas os respectivos pesos das ações do portifólio ótimo:


In [17]:
port_weights = opt2018[2:19]

Criamos uma fórmula semelhante, mas utilizando os pesos do portifólio ótimo de 2018, para obter a relação retorno risco desta alocação nos demais anos:


In [18]:
def result(ano):
  pf_data = pd.DataFrame()
  for s in stocks:
      end = str(ano)+'-12-31'
      start = str(ano)+'-01-01'
      pf_data[s] = yf.Ticker(s).history(start=start,end=end)['Close']
  ind_er = np.log(pf_data/pf_data.shift(1)).mean()*250
  port_er = (port_weights*ind_er).sum() #Observe que utilizamos a lista com os pesos do portifólio ótimo
  cov_matrix = pf_data.pct_change().apply(lambda x: np.log(1+x)).cov()
  var = cov_matrix.mul(port_weights, axis=0).mul(port_weights, axis=1).sum().sum() #Aqui também utilizamos a lista com os pesos do portifólio ótimo
  sd = np.sqrt(var) 
  ann_sd = sd*np.sqrt(250) 
  sharpe = (port_er-rf)/ann_sd
  return sharpe

Utilizamos a fórmula que acabamos de criar para encontrar a relação retorno risco do nosso portifólio ótimo para 2019:


In [19]:
result(2019)

2.258593197511229

Encontramos a relação de possíveis alocações para 2019 e ordenamos de acordo com o Índice de Sharpe:

In [20]:
base2019 = ports(2019)
base2019 = pd.DataFrame(sorted((base2019['Returns']-rf)/base2019['Volatility'])) 
base2019.iloc[94999] #Conseguimos encontrar também o valor que define a relação retorno risco das 5% melhores alocações para 2019

0    2.550996
Name: 94999, dtype: float64

Verificamos então se a relação retorno risco (sharpe ratio) do nosso portifólio ótimo de 2018 figura entre as 5% melhores alocações para 2019:


In [23]:
result(2019)>= base2019.iloc[94999]

0    False
Name: 94999, dtype: bool

Como o resultado foi negativo, podemos tentar ser menos exigentes e verificar se a relação figura entre as 20% melhores alocações para 2019:


In [27]:
result(2019)>= base2019.iloc[79999]

0    False
Name: 79999, dtype: bool

Para ser mais preciso, verificaremos a porcentagem exata de portifólios aleatórios que a nossa alocação ótima de 2018 consegue superar em 2019:


In [66]:
a = result(2019)>= base2019
a.describe()

,0
count,100000
unique,2
top,True
freq,50567


In [72]:
porcentagem = (a.describe()[0]['freq'])/(a.describe()[0]['count'])
print('Nosso portifólio ótimo de 2018 consegue superar '+str(round(porcentagem*100))+'% dos portifólios aleatórios de 2019')

Nosso portifólio ótimo consegue superar 51% dos portifólios aleatórios de 2019


#**CONCLUSÕES**


<p> Através da nossa análise, conseguimos observar que a alocação que maximiza o Índice de Sharpe para 2018 não apresenta o mesmo desempenho em 2019. O arcabouço utilizado pode ser facilmente replicado para outros anos e análises mais profundas, mas o que as informações sugerem para o período em questão é que o Índice de Sharpe, apesar de ser um ótimo mecanismo para a análise do desempenho passado de um portifólio, não é um parâmetro suficientemente bom para basear uma alocação futura.</p>  